# BW66 재사용 가능한 try,finally 동작을 원한다면 contextlib과 with문을 사용하라

## 🎨 사전지식

- with문
  - [구문] `with 객체 (as ALIAS) : 본문`
    - 단, 이때 객체는 \_\_enter\_\_, \_\_exit\_\_ 메소드를 포함해야 함
  - [의미] with문 안의 본문은 **객체**의 *context*를 불러와서 실행(\_\_enter\_\_)되고 with문 종료 시 자동으로 종료(\_\_exit\_\_)한다.

- *context*:  [컨텍스트란 정지된 것이 아니라 끊임없이 흘러가는 하나의 상태(status)이다.](https://organicmedia2.pressbooks.com/chapter/4-elements-of-context/)이다.
- status: 객체의 저장 상태

In [16]:
!echo "# it's test data" > test.txt

In [17]:
with open('test.txt') as f :
    content = f.read()
    print(content)

"# it's test data" 



### 📑 with와 사용자 설정 함수/객체의 쉬운 사용법: contextlib

In [20]:
import logging

def my_function():
    logging.debug('1: 디버깅 데이터')       #프로그램의 디폴트 로그 수준은 WARNING
    logging.error('2: 이 부분은 오류 로그')
    logging.debug('3: 추가 디버깅 데이터')

In [21]:
my_function()

ERROR:root:2: 이 부분은 오류 로그


In [28]:
from contextlib import contextmanager

@contextmanager
def debug_logging(level):
    logger = logging.getLogger()
    old_level = logger.getEffectiveLevel()
#     print(old_level)
    logger.setLevel(level)
#     print(logger.getEffectiveLevel())
    try:
        yield                            # with 블록의 내용이 실행되는 부분을 지정
    finally:
        logger.setLevel(old_level)

In [29]:
with debug_logging(logging.DEBUG):
    my_function()

DEBUG:root:1: 디버깅 데이터
ERROR:root:2: 이 부분은 오류 로그
DEBUG:root:3: 추가 디버깅 데이터


In [23]:
my_function()

ERROR:root:2: 이 부분은 오류 로그


### 📑 with와 대상변수(컨텍스트 객체) 함께 사용하기

In [26]:
@contextmanager
def log_level(level, name):
    logger = logging.getLogger(name)
    old_level = logger.getEffectiveLevel()
#     print(old_level)
    logger.setLevel(level)
#     print(logger.getEffectiveLevel())
    try:
        yield logger                      # with 블록의 내용이 실행되는 부분을 지정
    finally:
        logger.setLevel(old_level)  

In [27]:
with log_level(logging.DEBUG, 'my-log') as logger:
    logger.debug(f'대상: {logger.name}!')
    logging.debug('이 메세지는 출력되지 않습니다.')

DEBUG:my-log:대상: my-log!


# 참고자료

- [파이썬의 컨텍스트 매니저](https://sjquant.tistory.com/12)
- [컨텍스트란 정지된 것이 아니라 끊임없이 흘러가는 하나의 상태(status)](https://organicmedia2.pressbooks.com/chapter/4-elements-of-context/)